# Calculating Features

Calculating features is done through the `Atoms` class, which contains a molecular geometry. This is the central class used throughout ichor that deals with manipulating molecular geometries. Files typically have the `.atoms` attribute if they contain a molecular geometry.

In [3]:
from ichor.core.atoms import Atoms, Atom

# make some random geometry
atoms_instance = Atoms([Atom("O", 0.0, 0.0, 0.0),
                        Atom("H", 2.1, 2.3, 0.0),
                        Atom("H", 5.0, 0.0, 4.3)])

print(atoms_instance)

O1       0.00000000      0.00000000      0.00000000
H2       2.10000000      2.30000000      0.00000000
H3       5.00000000      0.00000000      4.30000000


## Calculators

Calculator functions take in an `Atom` instance and calculate something with it. An example below calculated the atomic local frame (ALF) features.

Features are calculated using feature calculator functions, which take in an `Atom` instance and return the calculated features. The calculators might need extra arguments, which are passed in as arguments or key word arguments. Currently, only an atomic local frame calculator is implemented, but there is nothing stopping a user from implementing their own calculators as needed.

In [7]:
from ichor.core.calculators import default_alf_calculator, default_feature_calculator
from pprint import pprint

# for the alf features, we need to define an ALF first
alf_dict = atoms_instance.alf_dict(default_alf_calculator)
pprint(alf_dict)

alf_list = atoms_instance.alf_list(default_alf_calculator)
pprint(alf_list)

{'H2': ALF(origin_idx=1, x_axis_idx=0, xy_plane_idx=2),
 'H3': ALF(origin_idx=2, x_axis_idx=0, xy_plane_idx=1),
 'O1': ALF(origin_idx=0, x_axis_idx=1, xy_plane_idx=2)}
[ALF(origin_idx=0, x_axis_idx=1, xy_plane_idx=2),
 ALF(origin_idx=1, x_axis_idx=0, xy_plane_idx=2),
 ALF(origin_idx=2, x_axis_idx=0, xy_plane_idx=1)]


As seen above, we first calculate the ALF for every atom. There is the `alf_list` and `alf_dict` methods, which return either a list of a dictionary respectively. In both cases, the `ALF` class is used to contain information about the atom's ALF. This `ALF` class is simply a named tuple containing the origin index, x axis index, and xy plane index. Note that these are 0-indexed as Python is 0 indexed. However, do note that the actual atom names (eg. O1, H2, etc.) do start at 1. There is no atom `H0` for example.

In [9]:
# calculating features with given alf

features = atoms_instance.features(default_feature_calculator, alf_dict)
pprint(features)

features = atoms_instance.features(default_feature_calculator, alf_list)
pprint(features)

array([[ 5.88551857, 12.46216712,  1.0341914 ],
       [ 5.88551857, 10.72159395,  1.61608526],
       [12.46216712, 10.72159395,  0.49131599]])
array([[ 5.88551857, 12.46216712,  1.0341914 ],
       [ 5.88551857, 10.72159395,  1.61608526],
       [12.46216712, 10.72159395,  0.49131599]])


The ALF features for the given `Atoms` instance are calculated as a numpy array of shape `natoms x nfeatures`. Since these features are per-atom, each atom has its own feature set. Now, we can also calculate the features for an individual atom like so

In [10]:
one_atom_features = atoms_instance["O1"].features(default_feature_calculator, alf_dict)
pprint(one_atom_features)

array([ 5.88551857, 12.46216712,  1.0341914 ])


This is done by directly calculating the ALF features only for the O1 atom.

## Defining a custom feature calculator

Let's say that you want to implement a custom calculator. All you need to do is implement a function that takes in an `Atom` instance and does something with it. Note that typically you will access the `parent` attribute of `Atom`, which is the `Atoms` instance containing the whole geometry.

In [14]:
import numpy as np

def x_coordinate_features(atom_instance: Atom):
    """A calculator function that grabs only the x-coordinate of all atoms.

    :param atom_instance: an Atom instance to work with
    :return: An array of x coordinates for all atoms in the system.
    """

    return np.array([a.coordinates[0] for a in atom_instance.parent])

new_set_of_features = atoms_instance["O1"].features(x_coordinate_features)

print(new_set_of_features)

[0.  2.1 5. ]


Then to get the features for all atoms, we go to the `Atoms` class instead, which just gets the features for all `Atom` instances held inside the `Atoms` class. Note that due to the simple calculator, the features for all atoms will be the same in this case.

In [15]:
new_set_of_features_all_atoms = atoms_instance.features(x_coordinate_features)

print(new_set_of_features_all_atoms)

[[0.  2.1 5. ]
 [0.  2.1 5. ]
 [0.  2.1 5. ]]
